# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:

import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




##  Preprocessing
- Lookup Table
- Tokenize Punctuation

- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`


In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text).keys()
    int_to_vocab = {ii: word for ii, word in enumerate(word_counts)}
    vocab_to_int = {word: ii for ii, word in enumerate(word_counts)}
    return vocab_to_int, int_to_vocab 

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

In [4]:
def token_lookup():
    return {'.': '||PERIOD||',         ',': '||COMMA||',            '"': '||QUOTATION_MARK||',
            ';': '||SEMICOLON||',      '!': '||EXCLAMATION_MARK||', '?': '||QUESTION_MARK||', 
            '(': '||LEFT_PAREN||',     ')': '||RIGHT_PAREN||',     '--': '||DASH||',
            '?': '||QUESTION_MARK||', '\n': '||RETURN||'}
 

tests.test_tokenize(token_lookup)

Tests Passed


helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [6]:

import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Neural Network
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

In [7]:

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
- Targets placeholder
- Learning Rate placeholder

Return the placeholders tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs  = tf.placeholder(tf.int32, [None,None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, None , name='lr')

    
    return inputs, targets, learning_rate

tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize


In [106]:
def get_init_cell(batch_size, rnn_size):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.85)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * 2)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    initial_state = tf.identity(initial_state, name='initial_state') 
    return cell, initial_state

tests.test_get_init_cell(get_init_cell)

Tests Passed


In [107]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)) ) 
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


tests.test_get_embed(get_embed)

Tests Passed


In [108]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(state, name='final_state')
    return outputs, final_state


tests.test_build_rnn(build_rnn)

Tests Passed


In [109]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    print(rnn_size,vocab_size, embed_dim, input_data.shape)
    
    embed = get_embed(input_data, vocab_size, embed_dim )
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, 
                                               vocab_size,
                                               weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer(),
                                               activation_fn=None)
    return logits, final_state


tests.test_build_nn(build_nn)

256 27 300 (128, 5)
Tests Passed


### Batches
 batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`


In [94]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """

    batch_count = len(int_text) // (batch_size * seq_length)
  
    xdata = np.array(int_text[: batch_count * batch_size * seq_length])
    ydata = np.array(int_text[1: batch_count * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), batch_count, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), batch_count, 1)
    
    return np.array(list(zip(x_batches, y_batches)))


tests.test_get_batches(get_batches)

Tests Passed


In [111]:
# Number of Epochs
num_epochs = 100 
# Batch Size
batch_size = 512 #floydhub gpu
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 100 #initial suggested in lesson 3.3
# Sequence Length
seq_length = 12 #~ avg number of lines/scene
# Learning Rate
learning_rate = 0.005  #tuned 
# Show stats for every n number of batches
show_every_n_batches = 10
#droppout on LSTM (in code) = 0.75

save_dir = './save'

In [112]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

512 6779 100 (?, ?)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [113]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/11   train_loss = 8.827
Epoch   0 Batch   10/11   train_loss = 6.647
Epoch   1 Batch    9/11   train_loss = 6.026
Epoch   2 Batch    8/11   train_loss = 6.022
Epoch   3 Batch    7/11   train_loss = 5.987
Epoch   4 Batch    6/11   train_loss = 5.818
Epoch   5 Batch    5/11   train_loss = 5.610
Epoch   6 Batch    4/11   train_loss = 5.339
Epoch   7 Batch    3/11   train_loss = 5.237
Epoch   8 Batch    2/11   train_loss = 4.962
Epoch   9 Batch    1/11   train_loss = 4.932
Epoch  10 Batch    0/11   train_loss = 4.796
Epoch  10 Batch   10/11   train_loss = 4.716
Epoch  11 Batch    9/11   train_loss = 4.577
Epoch  12 Batch    8/11   train_loss = 4.507
Epoch  13 Batch    7/11   train_loss = 4.452
Epoch  14 Batch    6/11   train_loss = 4.302
Epoch  15 Batch    5/11   train_loss = 4.212
Epoch  16 Batch    4/11   train_loss = 4.091
Epoch  17 Batch    3/11   train_loss = 4.056
Epoch  18 Batch    2/11   train_loss = 3.918
Epoch  19 Batch    1/11   train_loss = 3.878
Epoch  20 

In [114]:
helper.save_params((seq_length, save_dir))

In [115]:

import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [103]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    i_t = loaded_graph.get_tensor_by_name("input:0")
    i_s = loaded_graph.get_tensor_by_name("initial_state:0")
    f_s = loaded_graph.get_tensor_by_name("final_state:0")
    p = loaded_graph.get_tensor_by_name("probs:0")
 
    return i_t, i_s, f_s, p


tests.test_get_tensors(get_tensors)

Tests Passed


In [104]:
def pick_word(probabilities, int_to_vocab):
    return np.random.choice(list(int_to_vocab.values()), 10, p=probabilities)[0]


tests.test_pick_word(pick_word)

Tests Passed


In [117]:
gen_length = 200
# homer_simpson, moe_szyslak,  Barney_Gumble
prime_word = 'moe_szyslak'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        #print(np.sort(probabilities))
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: livin' business pour inspector nursemaid struggling xanders nasty confession bloodball feast shades nursemaid nursemaid nursemaid trip freed says fudd squeezin' tow-joes anguished strokkur inspector nursemaid struggling bloodball badge remain shades strokkur africa huddle neil_gaiman: kahlua log saw department's senators: confession nursemaid table's bloodball couch shades work fudd homer_doubles: log sharity inspector buddy log gentleman's refreshingness errrrrrr fudd harder part miles strokkur ahh errrrrrr weren't settles miserable confession nursemaid table's factor log buddy log gentleman's necessary shyly worst fudd shhh morning-after thousand-year confession focused nemo solid 'n' errrrrrr turkey errrrrrr man_at_bar: dank annie fudd breaking log weren't smiled chug fistiana confession disco_stu: shakes weren't j confession strokkur eva barflies: jumps confession bloodball talked chosen shades nursemaid managed character log neil_gaiman: gloop weren't fl errrrrrr poly